In [1]:
import torch
import os
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from MelodyLSTMEmbV2 import MelodyLSTMEmb

# Getting data

In [2]:
class ChordMelodyDataset(torch.utils.data.Dataset):
    def __init__(self, track_count, compass_count_path, X_path, y_path, window_size=10):
        self.window_size = window_size
        
        print('Loading compass count')
        self.compass_count = pd.read_csv(compass_count_path, names=['tid', 'compass_count'], nrows=track_count)
        self.cc = self.compass_count['compass_count'].to_numpy()
        self.cc_w = self.compass_count['compass_count'].to_numpy() - window_size
        self.cum_cc = np.cumsum(self.cc)
        self.cum_cc_w = np.cumsum(self.cc_w)
        
        self.cum_cc = np.append([0], self.cum_cc)
        self.cum_cc_w = np.append([0], self.cum_cc_w)

        
        print('Loading X')
        self.X = pd.read_csv(X_path, names=['n'+str(i) for i in range(36)] + ['chord', 'prev'], nrows=self.cum_cc[-1])
        
        print('Loading y')
        self.y = pd.read_csv(y_path, names=['n'+str(i) for i in range(36)], nrows=self.cum_cc[-1])
        
    def __len__(self):
        return self.cum_cc_w[-1]
    
    def __getitem__(self, idx):
        bucket = np.searchsorted(self.cum_cc_w, idx, side='right')
        # print(bucket)
        delta = idx - self.cum_cc_w[bucket - 1]
        # print(delta)
        idx_start = self.cum_cc[bucket - 1]
        # print(idx_start)
        idx = idx_start + delta
        
        # print(idx)
        x_data = torch.tensor(self.X.iloc[idx : idx + self.window_size].to_numpy(dtype=float))
        y_data = torch.tensor(self.y.iloc[idx : idx + self.window_size].to_numpy(dtype=float))

        return x_data.float(), y_data.float()

# Training

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device", device)

Device cuda


In [4]:
dataset = ChordMelodyDataset(100, '../data/compass_count2.csv', '../data/X2.csv', '../data/y2.csv', window_size=20)

Loading compass count
Loading X
Loading y


In [5]:
len(dataset)

77280

In [13]:
dataset[20]

(tensor([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
          0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
          0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
          0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 2.],
         [1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
   

In [14]:
data_loader = torch.utils.data.DataLoader(dataset, batch_size=256)

In [15]:
num_epochs = 100
learning_rate = 1e-6

input_size = 56 #number of features
hidden_size = 512 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers

output_size = 36 #number of output classes 

In [16]:
mlstm = MelodyLSTMEmb(input_size, hidden_size, output_size, 
                   num_layers, device, threshold=0.6)
mlstm = mlstm.to(device)

In [17]:
criterion = torch.nn.BCELoss()
optimizer = torch.optim.AdamW(mlstm.parameters(), lr=learning_rate) 

In [18]:
for epoch in range(num_epochs):
    for X, y in tqdm(data_loader):
        X = X.cuda()
        y = y.cuda()
        outputs = mlstm.forward(X) #forward pass
        optimizer.zero_grad() #calculate the gradient, manually setting to 0
        
        # obtain the loss function
        # print(outputs.shape, y[:,-1,:].shape)
        loss = criterion(outputs, y[:,-1,:])

        loss.backward() #calculates the loss of the loss function

        optimizer.step() #improve from loss, i.e backprop
        
    torch.save(mlstm.state_dict(), f'../models/muse2_t10000_bce_w20/checkpoint-{epoch:02}.pth')    
    print("Epoch: %d, loss: %1.8f" % (epoch, loss.item())) 

100%|████████████████████████████████████████████████████████████████████████████| 30262/30262 [28:46<00:00, 17.53it/s]


Epoch: 0, loss: 0.30035716


100%|████████████████████████████████████████████████████████████████████████████| 30262/30262 [28:03<00:00, 17.98it/s]


Epoch: 1, loss: 0.29951110


100%|████████████████████████████████████████████████████████████████████████████| 30262/30262 [27:09<00:00, 18.58it/s]


...

Epoch: 63, loss: 0.13565961


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30262/30262 [26:10<00:00, 19.27it/s]


Epoch: 64, loss: 0.13497289


 37%|██████████████████████████████████████████████████████▍                                                                                              | 11046/30262 [09:34<16:38, 19.24it/s]


KeyboardInterrupt: 

In [ ]:
torch.save(mlstm.state_dict(), '../models/muse2_t2000_v1.pth')

In [ ]:
mlstm.embedding(torch.tensor([1]).cuda())

In [ ]:
with open('../data/chords/CHORD_DICT.pickle', 'rb') as f: 
    CHORD_DICT = pickle.load(f)

In [ ]:
with open('../data/chords_reduced/CHORD_TO_EMB.pickle', 'rb') as f:
    CHORD_TO_EMB = pickle.load(f)

In [ ]:
CHORD_TO_EMB

In [ ]:
embeddings = mlstm.embedding.weight.data.cpu().numpy()

In [ ]:
embeddings[51]